# Лабораторная работа 7

   по теме
                 
   **Классификация обзоров фильмов**
   
   ****

   Выполнил студент

   Группы БСТ1801

   Харатишвили Заза

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Model, Input
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint
from pydash import *
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Flatten, Embedding, LSTM, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Average, Input, GlobalAveragePooling2D, Activation
import glob
import os

In [2]:
# Ограничим набор данных до 5000
top_words = 5000
# Устанавливаем максимальную длину
max_review_length = 500
# Вектор длины представления каждого слова
embedding_vector_length = 32
# Количество эпох
num_epochs = 3

In [3]:
# Загружаем датасет
from keras.datasets import imdb

(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=top_words)
# Обьединяем данные в доль одной оси
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

In [4]:
# Дополняем или сокращаем обзоры чтобы они были одинаковой длины
x_train = sequence.pad_sequences(training_data, maxlen=max_review_length)
x_test = sequence.pad_sequences(testing_data, maxlen=max_review_length)

y_train = training_targets
y_test = testing_data

x_test = np.asarray(x_train).astype(np.float32)
y_test = np.asarray(y_train).astype(np.float32)

# Задаем input_shape
input_shape = x_train[0,:].shape
# Входной слой модели
inp = Input(shape=input_shape)

In [5]:
model = Sequential()
# Embedding получает номера слов, а на выходе выдаёт их векторные представления
# Индекс слова не больше top_words-1
# Размерность векторного пространства embedding_vecor_length
# Число входов = max_review_length
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
# 100 нейронов (умных единиц памяти)
model.add(LSTM(100))
# На выходе 0 или 1, тк задача классификации
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=num_epochs, batch_size=64)
# Оценим качество модели по тестовым данным
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 100)               53200     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
391/391 [==============================] - 140s 355ms/step - loss: 0.4943 - accuracy: 0.7633 - val_loss: 0.3145 - val_accuracy: 0.8790
Epoch 2/3
391/391 [==============================] - 145s 372ms/step - loss: 0.3070 - accuracy: 0.8772 - val_loss: 0.2282 - val_accuracy: 0.9182
Epoch 3/3
391/391 [

In [6]:
# Добавим свертывающий слой и слой пула
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=num_epochs, batch_size=64)
# Оценим качество модели по тестовым данным
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 32)           160000    
                                                                 
 conv1d (Conv1D)             (None, 500, 32)           3104      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 250, 32)          0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
________________________________________________

Для того, чтобы провести ансамблирование моделей выведем модели в функции

In [7]:
# Функция компиляции и обучения
def compile_and_train(model, num_epochs): 
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    # Сохраняем модель
    checkpoint = ModelCheckpoint(filepath, 
                                 monitor='loss',  # Отслеживаем метрики потери
                                 verbose=0, 
                                 save_weights_only=True, # Сохраняем только веса модели
                                 save_best_only=True, # Сохраняем только лучшую модель
                                 mode='auto', period=1)
    history = model.fit(x=x_train, y=y_train, 
                        batch_size=32, epochs=num_epochs, 
                        verbose=1, callbacks=[checkpoint], 
                        validation_split=0.2)
    
    weight_files = glob.glob(os.path.join(os.getcwd(), '*'))
    weight_file = max(weight_files, key=os.path.getctime)
    
    print(model.evaluate(x_test, testing_targets, verbose=0))
    return history, weight_file

In [8]:
def lstm(model_input):

    emb = Embedding(top_words, embedding_vector_length)(model_input)
    lstm = LSTM(128)(emb)
    drop = Dropout(0.5)(lstm)
    dense = Dense(1)(drop)
    model_out = Activation(activation='sigmoid')(dense)
    
    model = Model(model_input, model_out, name='lstm')
    
    return model

model_v1 = lstm(Input(shape=x_train[0,:].shape))

In [9]:
_, lstm_weight_file = compile_and_train(model_v1, num_epochs)

Epoch 1/3
625/625 [==============================] - 90s 142ms/step - loss: 0.4896 - accuracy: 0.7631 - val_loss: 0.3477 - val_accuracy: 0.8568
Epoch 2/3
625/625 [==============================] - 90s 144ms/step - loss: 0.3550 - accuracy: 0.8497 - val_loss: 0.3767 - val_accuracy: 0.8402
Epoch 3/3
625/625 [==============================] - 89s 143ms/step - loss: 0.2776 - accuracy: 0.8895 - val_loss: 0.3628 - val_accuracy: 0.8388
[1.4356050491333008, 0.49851998686790466]


In [10]:
def cnn(model_input):

    emb = Embedding(top_words, embedding_vector_length, input_length=max_review_length)(model_input)

    conv1d_1 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(emb)
    conv1d_2 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(conv1d_1)
    conv1d_3 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(conv1d_2)
    mp1d_1 = MaxPooling1D(pool_size=2)(conv1d_3)
    drop_1 = Dropout(0.4)(mp1d_1)

    conv1d_4 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(drop_1)
    conv1d_5 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(conv1d_4)
    conv1d_6 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(conv1d_5)
    mp1d_2 = MaxPooling1D(pool_size=2)(conv1d_6)
    drop_2 = Dropout(0.4)(mp1d_2)
    lstm = LSTM(100)(drop_2)
    dense = Dense(1)(lstm)

    model_out = Activation(activation='sigmoid')(dense)
    
    model = Model(model_input, model_out, name='cnn')
    
    return model

model_v2 = cnn(Input(shape=x_train[0,:].shape))

In [11]:
_, cnn_weight_file = compile_and_train(model_v2, num_epochs)

Epoch 1/3
625/625 [==============================] - 65s 103ms/step - loss: 0.6933 - accuracy: 0.5027 - val_loss: 0.6935 - val_accuracy: 0.4938
Epoch 2/3
625/625 [==============================] - 64s 103ms/step - loss: 0.6932 - accuracy: 0.5002 - val_loss: 0.6931 - val_accuracy: 0.5062
Epoch 3/3
625/625 [==============================] - 64s 102ms/step - loss: 0.6933 - accuracy: 0.4983 - val_loss: 0.6932 - val_accuracy: 0.4938
[0.6931524276733398, 0.5]


In [12]:
# Сохраняем веса моделей
CNN_WEIGHT_FILE = os.path.join(cnn_weight_file)
LSTM_WEIGHT_FILE = os.path.join(lstm_weight_file)

# Получаем модели
cnn_model = cnn(inp)
lstm_model = lstm(inp)

# Загружаем сохраненные веса моделей
cnn_model.load_weights(CNN_WEIGHT_FILE)
lstm_model.load_weights(LSTM_WEIGHT_FILE)

# Обьединяем модели в один массив
models = [cnn_model, lstm_model]

In [13]:
def ansemble(models, model_input):
    # Получаем результаты моделей
    outputs = [model.outputs[0] for model in models]
    # Усредняем
    y = Average()(outputs)
    # Записываем в одну модель усредненные результаты работы моделей
    model = Model(model_input, y, name='ansemble')
    
    return model

In [14]:
# Обучаем модель, полученную после ансамблирования моделей
e_model = ansemble(models, inp)
e_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
e_model.fit(np.asarray(x_train).astype('float32'),
            np.asarray(y_train).astype('float32'),
            validation_data=(np.asarray(x_test).astype('float32'), np.asarray(y_train).astype('float32')),
            epochs=num_epochs, batch_size=64)

Epoch 1/3
391/391 [==============================] - 344s 876ms/step - loss: 0.3956 - accuracy: 0.9010 - val_loss: 0.3716 - val_accuracy: 0.9247
Epoch 2/3
391/391 [==============================] - 348s 890ms/step - loss: 0.3279 - accuracy: 0.9122 - val_loss: 0.2166 - val_accuracy: 0.9282
Epoch 3/3
391/391 [==============================] - 358s 915ms/step - loss: 0.2301 - accuracy: 0.9161 - val_loss: 0.1791 - val_accuracy: 0.9357


In [33]:
import string

# Получаем индексы из датасета
indexes = imdb.get_word_index()

# Функция, позволяющая загружать пользовательский текст и получать результат ансабля сетей
def user_text_predict(text, model):
    text_vec = np.zeros((1, max_review_length)) # Заполняем до max_review_length одномерный вектор
    # Преобразуем текст
    for idx, word in enumerate(text.translate(str.maketrans('', '', string.punctuation)).lower().split(' ')):
        # Если индексы больше 10000 или None присваиваем 0
        if idx < 500 and word in indexes:
            text_vec[0][idx] = indexes[word]
    tps = sequence.pad_sequences(text_vec, maxlen=max_review_length)
    return np.around(model.predict(tps))

In [37]:
user_text_predict('This film is disgusting', e_model)

array([[0.]], dtype=float32)

## Вывод

В результате выполнения лабораторной работы было проведено ансамблирование моделей, благодаря которому значительно повысилась точность работы. Была написана функция, позволяющая загружать пользовательский текст и получать результат ансабля сетей, и провели тестирование сетей на основе загруженного текста.